In [1]:
import numpy as np
from scipy.integrate import quad

# Initial definitions
I_0 = 1.5  # Initial value for I
K_0 = 2.0  # Initial value for K
T = 1.0    # Terminal time
N = 1000   # Number of time steps
M = 100000  # Number of simulations
dt = T / N # Time step size

# Define the time-dependent functions
def mu(t):
    return 0.005 * t + 0.01 * np.sin(t)

def sigma(t):
    return 0.2 + 0.05 * np.cos(t)

def gamma(t):
    return np.sin(t)

# Theoretical calculations
def E_gL(t):
    integrand = lambda s: I_0 * gamma(s) * mu(s) * np.exp(quad(lambda u: K_0 * gamma(u) * mu(u), s, t)[0])
    return quad(integrand, 0, t)[0]

def E_gS(t):
    integrand = lambda s: -I_0 * gamma(s) * mu(s) * np.exp(-quad(lambda u: K_0 * gamma(u) * mu(u), s, t)[0])
    return quad(integrand, 0, t)[0]

def Xi_L(t):
    return 2 * K_0 * gamma(t) * mu(t) + K_0**2 * gamma(t)**2 * sigma(t)**2

def Psi_L(t):
    return 2 * (I_0 * gamma(t) * mu(t) + I_0 * K_0 * gamma(t)**2 * sigma(t)**2) * E_gL(t) + I_0**2 * gamma(t)**2 * sigma(t)**2

def Xi_S(t):
    return -2 * K_0 * gamma(t) * mu(t) + K_0**2 * gamma(t)**2 * sigma(t)**2

def Psi_S(t):
    return 2 * (-I_0 * gamma(t) * mu(t) + I_0 * K_0 * gamma(t)**2 * sigma(t)**2) * E_gS(t) + I_0**2 * gamma(t)**2 * sigma(t)**2

def E_gL2(t):
    integrand = lambda s: Psi_L(s) * np.exp(quad(Xi_L, s, t)[0])
    return quad(integrand, 0, t)[0]

def E_gS2(t):
    integrand = lambda s: Psi_S(s) * np.exp(quad(Xi_S, s, t)[0])
    return quad(integrand, 0, t)[0]

def A(t):
    return -gamma(t) * mu(t) * I_0 * (E_gL(t) - E_gS(t)) - gamma(t)**2 * sigma(t)**2 * (I_0**2 + I_0 * K_0 * (E_gL(t) + E_gS(t)))

def B(t):
    return -gamma(t)**2 * sigma(t)**2 * K_0**2

def E_gLgS(t):
    integrand = lambda s: A(s) * np.exp(quad(B, s, t)[0])
    return quad(integrand, 0, t)[0]

def Var_gL_plus_gS(t):
    return E_gL2(t) - E_gL(t)**2 + E_gS2(t) - E_gS(t)**2 + 2 * E_gLgS(t) - 2 * E_gL(t) * E_gS(t)

# Monte Carlo simulation for g_L(t) and g_S(t)
def simulate_paths():
    dW = np.sqrt(dt) * np.random.randn(M, N)  # Brownian increments
    g_L = np.zeros((M, N+1))  # Matrix to hold long position paths
    g_S = np.zeros((M, N+1))  # Matrix to hold short position paths

    for i in range(1, N+1):
        t = i * dt
        g_L[:, i] = g_L[:, i-1] + (K_0 * gamma(t) * mu(t) * g_L[:, i-1] + I_0 * gamma(t) * mu(t)) * dt \
                     + (K_0 * gamma(t) * sigma(t) * g_L[:, i-1] + I_0 * gamma(t) * sigma(t)) * dW[:, i-1]
        g_S[:, i] = g_S[:, i-1] + (-K_0 * gamma(t) * mu(t) * g_S[:, i-1] - I_0 * gamma(t) * mu(t)) * dt \
                     + (-K_0 * gamma(t) * sigma(t) * g_S[:, i-1] - I_0 * gamma(t) * sigma(t)) * dW[:, i-1]

    return g_L, g_S

# Run the simulation
g_L, g_S = simulate_paths()
g_L_last = g_L[:, -1]
g_S_last = g_S[:, -1]

# Calculate empirical expectations and variances
E_g_L_empirical = np.mean(g_L_last)
E_g_S_empirical = np.mean(g_S_last)
E_g_L_squared_empirical = np.mean(g_L_last**2)
E_g_S_squared_empirical = np.mean(g_S_last**2)
E_g_LgS_empirical = np.mean(g_L_last * g_S_last)

Var_g_L_empirical = E_g_L_squared_empirical - E_g_L_empirical**2
Var_g_S_empirical = E_g_S_squared_empirical - E_g_S_empirical**2
Cov_g_L_g_S_empirical = E_g_LgS_empirical - E_g_L_empirical * E_g_S_empirical
Var_g_L_plus_g_S_empirical = Var_g_L_empirical + Var_g_S_empirical + 2 * Cov_g_L_g_S_empirical

# Calculate theoretical expectations and variances
E_g_L_theoretical = E_gL(T)
E_g_S_theoretical = E_gS(T)
E_g_L_squared_theoretical = E_gL2(T)
E_g_S_squared_theoretical = E_gS2(T)
E_g_LgS_theoretical = E_gLgS(T)
Var_g_L_theoretical = E_g_L_squared_theoretical - E_g_L_theoretical**2
Var_g_S_theoretical = E_g_S_squared_theoretical - E_g_S_theoretical**2
Var_g_L_plus_g_S_theoretical = Var_gL_plus_gS(T)

# Print theoretical and empirical results
print("Theoretical Results:")
print(f"E[g_L(T)]: {E_g_L_theoretical}")
print(f"E[g_S(T)]: {E_g_S_theoretical}")
print(f"E[g_L(T)^2]: {E_g_L_squared_theoretical}")
print(f"E[g_S(T)^2]: {E_g_S_squared_theoretical}")
print(f"E[g_L(T)g_S(T)]: {E_g_LgS_theoretical}")
print(f"Var[g_L(T)]: {Var_g_L_theoretical}")
print(f"Var[g_S(T)]: {Var_g_S_theoretical}")
print(f"Variance of g_L(T) + g_S(T): {Var_g_L_plus_g_S_theoretical}")

print("Monte Carlo Results:")
print(f"E[g_L(T)]: {E_g_L_empirical}")
print(f"E[g_S(T)]: {E_g_S_empirical}")
print(f"E[g_L(T)^2]: {E_g_L_squared_empirical}")
print(f"E[g_S(T)^2]: {E_g_S_squared_empirical}")
print(f"E[g_L(T)g_S(T)]: {E_g_LgS_empirical}")
print(f"Var[g_L(T)]: {Var_g_L_empirical}")
print(f"Var[g_S(T)]: {Var_g_S_empirical}")
print(f"Variance of g_L(T) + g_S(T): {Var_g_L_plus_g_S_empirical}")

Theoretical Results:
E[g_L(T)]: 0.006375848080755279
E[g_S(T)]: -0.006322103055908149
E[g_L(T)^2]: 0.036027358569246326
E[g_S(T)^2]: 0.034828536924717615
E[g_L(T)g_S(T)]: -0.033328942230404536
Var[g_L(T)]: 0.03598670713049745
Var[g_S(T)]: 0.034788567937668095
Variance of g_L(T) + g_S(T): 0.0041980081446271675
Monte Carlo Results:
E[g_L(T)]: 0.006631234979499813
E[g_S(T)]: -0.006549055087577699
E[g_L(T)^2]: 0.03606503701544172
E[g_S(T)^2]: 0.03489234393585726
E[g_L(T)g_S(T)]: -0.03338731584495466
Var[g_L(T)]: 0.036021063738088374
Var[g_S(T)]: 0.03484945381331713
Variance of g_L(T) + g_S(T): 0.004182742507855011


In [2]:
# Simplified closed-form solution of the variance
def calculate_variance(t):
    # Calculate the integrals
    integral_1, _ = quad(lambda tau: K_0**2 * gamma(tau)**2 * sigma(tau)**2, 0, t)
    integral_2, _ = quad(lambda tau: 2 * K_0 * mu(tau) * gamma(tau), 0, t)

    # Using integral_1 for both the e^(-integral) terms as they are the same
    # Calculate the main variance equation
    variance = (I_0**2 / K_0**2) * (np.exp(integral_1) - 1) * (np.exp(integral_2) + np.exp(-integral_2) - 2 * np.exp(-integral_1))

    return variance

# Calculate the variance at the terminal time T
variance_theoretical_simplified = calculate_variance(T)
print(f"Theoretical variance (simplified) of g_L(T) + g_S(T) at time T = {T}: {variance_theoretical_simplified}")

Theoretical variance (simplified) of g_L(T) + g_S(T) at time T = 1.0: 0.00419800814462717
